In [1]:
import os
import matplotlib.pyplot as plt

from enum import Enum
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import (
    Simulator,
    StartMeshcat,
)

from manipulation import FindResource, running_as_notebook
from manipulation.scenarios import AddMultibodyTriad
from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

import os
import numpy as np
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.robot.test_direct_joint_control import (
    TestDirectJointControl,
)
from manipulation.station import MakeHardwareStation, load_scenario

import numpy as np
from pydrake.all import (
    AbstractValue,
    ConstantVectorSource,
    DiagramBuilder,
    LeafSystem,
    DepthImageToPointCloud,
    PiecewisePose,
    RigidTransform,
    PiecewisePolynomial,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    RotationMatrix,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_robot_painter import TestRobotPainter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddIiwaDifferentialIK
from manipulation.station import MakeHardwareStation, load_scenario


from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    FindResourceOrThrow,
    Parser,
    RandomGenerator,
    SceneGraph,
    RigidTransform,
    Role,
    RollPitchYaw,
    Simulator,
    UniformlyRandomRotationMatrix,
    MakeRenderEngineVtk,
    RenderEngineVtkParams,
    RenderCameraCore,
    DepthRenderCamera,
    PointCloud,
    BaseField,
    Fields,
)

from pydrake.all import (
    AbstractValue, Adder, AddMultibodyPlantSceneGraph, BallRpyJoint, BaseField,
    Box, CameraInfo, ClippingRange,
    DepthImageToPointCloud, DepthRange, DepthRenderCamera, DiagramBuilder,
    FindResourceOrThrow,
    LeafSystem,
    MakeRenderEngineVtk, ModelInstanceIndex, Parser, RenderCameraCore,
    RenderEngineVtkParams, RgbdSensor, RigidTransform,
    RollPitchYaw, Role,
    ImageRgba8U, ImageDepth32F)

from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    load_scenario,
)

In [8]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
def convert_to_coords(pos):
    x = ord(pos[0]) - ord('a')
    y = int(pos[1])-1
    return "" + str(.6*.125*(y-3.5)) + ", "+ str(.6*.125*(x-3.5)+0.55) 

def get_coord_from_pos(pos):
    x = ord(pos[0]) - ord('a')
    y = int(pos[1])-1
    return [.6*.125*(y-3.5),.6*.125*(x-3.5)+0.55]

def get_starting_pose(pos):
    # They are flipped buy oh well
    x = ord(pos[0]) - ord('a')
    y = int(pos[1])-1
    return RigidTransform([.6*.125*(y-3.5), .6*.125*(x-3.5)+0.55, 0])

In [4]:
def get_scene():
    with open('board.yaml', 'r') as f:
        output = f.read()
    return output

In [5]:
def get_observation_pose(spot):
    coord = get_coord_from_pos(spot)
    if int(spot[1]) < 5:
        observe = RigidTransform(
                RollPitchYaw(-np.pi/4,0,np.pi/2),
                [coord[0]+0.15, coord[1], 0.15])
    else:
        observe = RigidTransform(
                RollPitchYaw(-np.pi/4,0,-np.pi/2),
                [coord[0]-0.15, coord[1], 0.15])
    return observe

def get_hover_pose(spot):
    coord = get_coord_from_pos(spot)
    if int(spot[1]) < 5:
        grab = RigidTransform(
            RollPitchYaw(-np.pi/3,0,np.pi/2),
            [coord[0]+0.0475, coord[1]-0.01, 0.2]
        )
    else:
        grab = RigidTransform(
            RollPitchYaw(-np.pi/3,0,-np.pi/2),
            [coord[0]-0.0475, coord[1]-0.01, 0.2]
        )
    return grab

def get_grab_pose(spot):
    coord = get_coord_from_pos(spot)
    if int(spot[1]) < 5:
        grab = RigidTransform(
            RollPitchYaw(-np.pi/3,0,np.pi/2),
            [coord[0]+0.0475, coord[1]-0.01, 0.12]
        )
    else:
        grab = RigidTransform(
            RollPitchYaw(-np.pi/3,0,-np.pi/2),
            [coord[0]-0.0475, coord[1]-0.01, 0.12]
        )
    return grab

def crop_cloud(cloud, pos):
    #compute crop box
    p_C = get_coord_from_pos(pos) + [0.0]
    X_C = RigidTransform(
            RollPitchYaw(0,0,0),
            p_C
        )
    a = X_C.multiply(
        [.3*.125, .3*.125, 0.012]
    )
    b = X_C.multiply(
        [-.3*.125, -.3*.125, 0.035]
    )
    _crop_lower = np.minimum(a, b)
    _crop_upper = np.maximum(a, b)

    cloud_of_interest = cloud.Crop(_crop_lower, _crop_upper)
    return cloud_of_interest.xyzs()

def get_pose_from_pcl(cloud, pos):
    cloud_in_W = crop_cloud(cloud, pos)
    x = cloud_in_W[0]
    y = cloud_in_W[1]
    y_C = np.mean(y)
    x_C = max(x, key=lambda p: np.abs(p))
    
    #now calculate grab
    if int(pos[1]) < 5:
        grab = RigidTransform(
            RollPitchYaw(-np.pi/3,0,np.pi/2),
            [x_C+0.0475, y_C-0.01, 0.12]
        )
    else:
        grab = RigidTransform(
            RollPitchYaw(-np.pi/3,0,-np.pi/2),
            [x_C-0.0475, y_C-0.01, 0.12]
        )
    return grab

def is_there_capture(cloud, pos):
    cloud_in_W = crop_cloud(cloud, pos)
    return False if len(cloud_in_W[0]) < 20 else True


In [ ]:
class ChessStates(Enum):
    INIT = -1
    OBSERVE_CAPTURE = 0
    CAPTURE = 1 #special state for capturing a piece
    OBSERVE_LOCATION = 2
    MOVE_PIECE = 3 #switch back to 1
    

class PoseTrajectorySource(LeafSystem):

    def __init__(self, plant, rob, traj=None, gtraj=None):
        LeafSystem.__init__(self)

        #robot trajectories
        self.robot = rob
        self._pose_trajectory = traj
        self._grip_trajectory = gtraj

        #gripper information
        self._gripper_body_index = plant.GetBodyByName("body").index()
        self.open_grip = 0.05
        self.close_grip = 0.00

        #state information
        self.STATE = ChessStates.INIT
        self.pose = None
        self.grip = np.array([[0.05]])
        self.plan = []
        self.plan_index = 0
        self.need_to_publish = True

        #constant frames
        self.X_init = RigidTransform(
                RollPitchYaw(-np.pi/2,0,0),
                [0.0004543007558398612, 0.5704952573763858, 0.2643136978639653]
        )

        #ports
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcPose
        )
        self.DeclareVectorOutputPort("grip", 1, self.CalcGrip)
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()])
        )
        model_point_cloud = AbstractValue.Make(PointCloud(0))
        self.DeclareAbstractInputPort("point_cloud", model_point_cloud)
        #might need to add plant input and also body poses for more robustness

    #update function, if the trajectory over, then we want to make a new trajectory to match the next step
    def updatePlan(self, newPlan):
        self.need_to_publish = True
        self.plan += newPlan

    def CalcPose(self, context, output):
        if self._pose_trajectory and self._pose_trajectory.is_time_in_range(context.get_time()):
            self.pose = self._pose_trajectory.GetPose(context.get_time())
        else: #not in a trajectory
            if not self._pose_trajectory or context.get_time() > self._pose_trajectory.end_time(): #need to recalcualte
                self.replan(context)
        if self.pose == None:
            self.pose = self.GetInputPort("body_poses").Eval(context)[int(self._gripper_body_index)]
        
        output.set_value(self.pose)

    def CalcGrip(self, context, output):
        if self._grip_trajectory and self._grip_trajectory.is_time_in_range(context.get_time()):
            self.grip = self._grip_trajectory.value(context.get_time())
            #update self.grip if there is a need to, otherwise hold position
        output.SetFromVector(self.grip)

    def replan(self, context):
        """
        Creates Trajectory plans for the robot to follow, once the trajectory is over, this is recalled
        """
        #check if there are things left to do 
        if self.plan_index >= len(self.plan):
            if self.need_to_publish and self.plan_index > 0:
                self.robot.visualizer.PublishRecording()
                self.need_to_publish = False
                print("reached")
                print(self.need_to_publish)
                print(self.plan_index)
            return

        #get current positions
        current_pose = self.GetInputPort("body_poses").Eval(context)[int(self._gripper_body_index)]
        current_grip = self.grip[0][0]
        plan_step = self.plan[self.plan_index]
        pick_spot = plan_step[0]
        place_spot = plan_step[1]

        #figure out the next move
        match self.STATE:
            case ChessStates.INIT:
                #planGrip
                key_grip_poses = np.array([current_grip, self.open_grip]).reshape(1, 2)
                #planPose
                key_frame_poses = [current_pose, self.X_init]
                #plan times
                pose_times = grip_times = [context.get_time(), context.get_time()+1]
            case ChessStates.OBSERVE_CAPTURE:
                #planGrip
                key_grip_poses = np.array([current_grip, self.open_grip]).reshape(1, 2)
                #planPose
                X_obs = get_observation_pose(place_spot)
                key_frame_poses = [current_pose, X_obs]
                #plan times
                pose_times = grip_times = [context.get_time(), context.get_time()+1.5]
            case ChessStates.CAPTURE: #clear the piece on the place spot
                cloud = self.GetInputPort("point_cloud").Eval(context)
                if not is_there_capture(cloud, place_spot):
                    self.STATE = ChessStates.OBSERVE_LOCATION
                    self.replan(context)
                    return
                #key frames
                #X_pick = get_grab_pose(pick_spot)
                X_place = get_pose_from_pcl(cloud, place_spot)
                #key frames
                X_place_hover = get_hover_pose(place_spot)
                X_end_throw = RigidTransform(RollPitchYaw(0,0,0), 
                                             [0, 0.75, 0.3])
                #plan frames
                key_frame_poses = [current_pose, 
                                   X_place_hover, 
                                   X_place,
                                   X_place,
                                   X_place_hover,
                                   X_end_throw]
                #planGrip
                key_grip_poses = np.array([current_grip, 
                                           self.open_grip,
                                           self.open_grip,
                                           self.close_grip,
                                           self.close_grip,
                                           self.open_grip]).reshape(1, 6)
                #planTimes
                pose_times = [context.get_time(),
                              context.get_time()+0.5,
                              context.get_time()+1.5,
                              context.get_time()+2,
                              context.get_time()+3,
                              context.get_time()+4]
                grip_times = [context.get_time(),
                              context.get_time()+0.5,
                              context.get_time()+1.5,
                              context.get_time()+2,
                              context.get_time()+3.95,
                              context.get_time()+4]
            case ChessStates.OBSERVE_LOCATION:
                #planGrip
                key_grip_poses = np.array([current_grip, self.open_grip]).reshape(1, 2)
                #planPose
                X_obs = get_observation_pose(pick_spot)
                key_frame_poses = [current_pose, X_obs]
                #plan times
                pose_times = grip_times = [context.get_time(), context.get_time()+1.5]
            case ChessStates.MOVE_PIECE:
                cloud = self.GetInputPort("point_cloud").Eval(context)
                #key frames
                #X_pick = get_grab_pose(pick_spot)
                X_pick = get_pose_from_pcl(cloud, pick_spot)
                X_pick_hover = get_hover_pose(pick_spot)
                X_place_hover = get_hover_pose(place_spot)
                X_place = get_grab_pose(place_spot)
                #plan frames
                key_frame_poses = [current_pose, 
                                   X_pick_hover, 
                                   X_pick,
                                   X_pick,
                                   X_pick_hover,
                                   X_place_hover,
                                   X_place,
                                   X_place,
                                   X_place_hover,]
                #planGrip
                key_grip_poses = np.array([current_grip, 
                                           self.open_grip,
                                           self.open_grip,
                                           self.close_grip,
                                           self.close_grip,
                                           self.open_grip]).reshape(1, 6)
                #planTimes
                pose_times = [context.get_time(),
                              context.get_time()+0.5,
                              context.get_time()+1.5,
                              context.get_time()+2,
                              context.get_time()+3,
                              context.get_time()+5,
                              context.get_time()+6,
                              context.get_time()+6.5,
                              context.get_time()+7]
                grip_times = [context.get_time(),
                              context.get_time()+0.5,
                              context.get_time()+1.5,
                              context.get_time()+2,
                              context.get_time()+6,
                              context.get_time()+6.5]
                self.plan_index += 1
                
            case _:
                #some unknown state
                print("UNKNOWN: GOING HOME")
                self.STATE = ChessStates.INIT
                self.replan(context)

        #make new traj
        self._pose_trajectory = PiecewisePose.MakeLinear(pose_times, key_frame_poses)
        self._grip_trajectory = PiecewisePolynomial.FirstOrderHold(grip_times, key_grip_poses)

        self.STATE = ChessStates((self.STATE.value + 1)%4)
        
class RobotPlayer:

    def __init__(self, traj = None):
        builder = DiagramBuilder()
        #pass in which piece is moving and weld the others
        scenario_data = get_scene()
        scenario = load_scenario(data=scenario_data)
        self.station = builder.AddSystem(
            MakeHardwareStation(scenario, meshcat=meshcat)
        )
        self.plant = self.station.GetSubsystemByName("plant")
        # Remove joint limits from the wrist joint.
        self.plant.GetJointByName("iiwa_joint_7").set_position_limits(
            [-np.inf], [np.inf]
        )
        controller_plant = self.station.GetSubsystemByName(
            "iiwa.controller"
        ).get_multibody_plant_for_control()
        # # Export the point cloud output

        # optionally add trajectory source
        # if traj is not None:
        self.MovePlanner = builder.AddSystem(PoseTrajectorySource(self.plant, self, traj))
        builder.Connect(
            self.station.GetOutputPort("body_poses"), self.MovePlanner.GetInputPort("body_poses")
        )
        self.controller = AddIiwaDifferentialIK(
            builder,
            controller_plant,
            frame=controller_plant.GetFrameByName("body"),
        )
        builder.Connect(
            self.MovePlanner.GetOutputPort("pose"),
            self.controller.get_input_port(0),
        )
        builder.Connect(
            self.station.GetOutputPort("iiwa.state_estimated"),
            self.controller.GetInputPort("robot_state"),
        )

        builder.Connect(
            self.controller.get_output_port(),
            self.station.GetInputPort("iiwa.position"),
        )

        builder.Connect(
            self.MovePlanner.GetOutputPort("grip"),
            self.station.GetInputPort("wsg.position"),
        )

        to_point_cloud = AddPointClouds(
            scenario=scenario, station=self.station, builder=builder
        )
        builder.Connect(
            to_point_cloud["camera0"].get_output_port(),
            self.MovePlanner.GetInputPort("point_cloud"),
        )


        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder,
            self.station.GetOutputPort("query_object"),
            meshcat,
            MeshcatVisualizerParams(delete_on_initialization_event = False),
        )
        
        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName("body")
        self.world_frame = self.plant.world_frame()

        self.context = self.CreateDefaultContext()

        self.visualizer.StartRecording(True)
        self.simulator = Simulator(self.diagram)

    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies

        Input:
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.

        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(
            meshcat, "painter/" + name, length=length, radius=radius, X_PT=X_WF
        )

    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context
        )

        # provide initial states
        q0 = np.array(
            [
                1.57,
                0.4,
                0,
                -1.2,
                0,
                1.5,
                -3.14,
            ]
        )
        # set the joint positions of the kuka arm
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.025, 0.025])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])

        return context

    def get_X_WG(self):
        plant_context = self.plant.GetMyMutableContextFromRoot(self.context)
        X_WG = self.plant.CalcRelativeTransform(
            plant_context, frame_A=self.world_frame, frame_B=self.gripper_frame
        )
        return X_WG

    #change to make move, feed it a move and it makes that one move
    def makeMove(self, newPlan):

        self.MovePlanner.updatePlan(newPlan)

        duration = 20*len(newPlan) if running_as_notebook else 0.01

        #move the meshcat to an attribute 
        self.simulator.AdvanceTo(self.context.get_time()+duration)

        self.visualizer.PublishRecording()

        #check camera, make new trajectories
        #find next trajectories
        #restart the robot 

In [32]:
meshcat.Delete()
painter = RobotPlayer()
plan = [('e2', 'e4'), ('g8', 'f6'), ('e1', 'e2'), ('f6', 'e4'), ('e4', 'g3')] 
painter.makeMove(plan)

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x557c44193610): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 272
vtkPNGReader (0x557c44193610): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x557c44193610): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 396
vtkPNGReader (0x557c44193610): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x557c44623a40): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 272
vtkPNGReader (0x557c44623a40): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x557c44623a40): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 396
vtkPNGReader (0x557c44623a40): Invalid file header: not a PNG file

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


ValueError: max() arg is an empty sequence

In [ ]:
plan = [('d2', 'e5')] 
painter.makeMove(plan)

20


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c8432d35-2370-4de9-924c-af3802cf315d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>